In [47]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import umap.umap_ as umap
from collections import Counter

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Define paths
XML_DIR = Path("./../3 GNN/xml_files")
MODEL_DIR = Path("./../2 second_classifier_premise-vs-conclusion/results/secondAttemptWithMetrics/RoBERTa_prem_conc_finetuned/")
OUTPUT_DIR = Path("classifier_results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load XML files
xml_files = [f for f in XML_DIR.glob("*.xml") if f.is_file()]
print(f"Found {len(xml_files)} XML files")


Using device: cuda
Found 39 XML files


In [48]:
def process_xml(xml_path):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        nodes = []
        id_registry = set()
        for elem in root.iter():
            if elem.tag not in ('prem', 'conc'):
                continue
                
            node_id = elem.attrib.get('ID', '').strip()
            if not node_id or node_id in id_registry:
                continue
                
            nodes.append({
                'id': node_id,
                'text': elem.text.strip() if elem.text else '',
                'type': elem.tag,
                'xml_file': xml_path.name
            })
            id_registry.add(node_id)
            
        id_to_idx = {node['id']: idx for idx, node in enumerate(nodes)}
        
        # Extract relations
        relations = []
        for elem in root.iter():
            if elem.tag not in ('prem', 'conc'):
                continue
                
            source_id = elem.attrib.get('ID', '').strip()
            if not source_id or source_id not in id_to_idx:
                continue

            def clean_split(value):
                return [t.strip() for t in value.strip().split('|') if t.strip()]
                
            for rel_type in ['SUP', 'ATT']:
                if rel_type in elem.attrib:
                    targets = clean_split(elem.attrib[rel_type])
                    relation_label = 0 if rel_type == 'SUP' else 1  # 0=Support, 1=Attack
                    
                    for target_id in targets:
                        if target_id in id_to_idx:
                            source_idx = id_to_idx[source_id]
                            target_idx = id_to_idx[target_id]
                            relations.append({
                                'source_id': source_id,
                                'target_id': target_id,
                                'source_idx': source_idx,
                                'target_idx': target_idx,
                                'label': relation_label,
                                'xml_file': xml_path.name
                            })
        
        return nodes, relations, id_to_idx
        
    except ET.ParseError as e:
        print(f"XML parse error in {xml_path.name}: {e}")
        return [], [], {}


In [49]:
def generate_no_relation_samples(nodes, relations, id_map):
    # Create set of existing relations
    existing_relations = set()
    for rel in relations:
        existing_relations.add((rel['source_idx'], rel['target_idx']))
    
    # Count support and attack relations
    support_count = sum(1 for rel in relations if rel['label'] == 0)
    attack_count = sum(1 for rel in relations if rel['label'] == 1)
    
    # Determine sample size based on class frequencies
    sample_size = support_count + attack_count * 5  # Prioritize attack relations
    
    # Generate all possible No-Relation pairs
    no_relation_candidates = []
    for i, source in enumerate(nodes):
        for j, target in enumerate(nodes):
            if i != j and (i, j) not in existing_relations:
                # Prioritize premise-conclusion pairs as they're more meaningful
                priority = 2 if source['type'] == 'prem' and target['type'] == 'conc' else 1
                no_relation_candidates.append({
                    'source_id': source['id'],
                    'target_id': target['id'],
                    'source_idx': i,
                    'target_idx': j,
                    'label': 2,  # 2=No-Relation
                    'priority': priority,
                    'xml_file': source['xml_file']
                })
    
    # Sort by priority and sample
    no_relation_candidates.sort(key=lambda x: x['priority'], reverse=True)
    sample_size = min(sample_size, len(no_relation_candidates))
    sampled_no_relations = no_relation_candidates[:sample_size]
    
    print(f"Sampled {len(sampled_no_relations)} No-Relation pairs from {len(no_relation_candidates)} candidates")
    return sampled_no_relations


In [50]:
# Load RoBERTa model

# PRETRAINED_MODEL_NAME = "roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
# model = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
# model = model.to(device)
# model.eval()


print("Loading RoBERTa model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModel.from_pretrained(MODEL_DIR)
model = model.to(device)
model.eval()

def generate_embeddings(texts, batch_size=8):
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch, 
            padding=True, 
            truncation=True, 
            max_length=512, 
            return_tensors="pt"
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get CLS token embeddings
        embeddings.append(outputs.last_hidden_state[:,0,:].cpu())
    
    return torch.cat(embeddings, dim=0)


def generate_embeddings_from_word_embeddings(texts, batch_size=8):
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch, 
            padding=True, 
            truncation=True, 
            max_length=512, 
            return_tensors="pt"
        ).to(device)
        
        # Extract token IDs from the tokenizer output
        input_ids = inputs['input_ids']
        
        with torch.no_grad():
            # Get the embeddings directly from the embedding layer
            word_embeddings = model.embeddings.word_embeddings(input_ids)
            
            # For each sequence, aggregate word embeddings (e.g., average over all tokens)
            # You can also use the embeddings of specific tokens (like [CLS]) if needed
            batch_embeddings = word_embeddings.mean(dim=1)  # Mean across all tokens
            embeddings.append(batch_embeddings.cpu())
    
    return torch.cat(embeddings, dim=0)



Some weights of RobertaModel were not initialized from the model checkpoint at ../2 second_classifier_premise-vs-conclusion/results/secondAttemptWithMetrics/RoBERTa_prem_conc_finetuned and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading RoBERTa model...


In [51]:
# Process all XML files first to gather argument pairs
all_nodes = []
all_relations = []
all_no_relations = []
doc_to_pairs = {}  # Track pairs by document

for xml_file in tqdm(xml_files, desc="Processing XML files"):
    nodes, relations, id_map = process_xml(xml_file)
    
    if not nodes:
        print(f"Skipping {xml_file.name}: No valid nodes found")
        continue
    
    # Generate no-relation samples
    no_relations = generate_no_relation_samples(nodes, relations, id_map)
    
    # Add to global collections
    start_idx = len(all_nodes)
    all_nodes.extend(nodes)
    all_relations.extend(relations)
    all_no_relations.extend(no_relations)
    
    # Store pairs for this document
    doc_pairs = []
    
    # Extract relation pairs
    for rel in relations:
        source_node = nodes[rel['source_idx']]
        target_node = nodes[rel['target_idx']]
        
        doc_pairs.append({
            'source_text': source_node['text'],
            'target_text': target_node['text'],
            'label': rel['label'],  # 0=Support, 1=Attack
            'document': xml_file.name
        })
    
    # Extract no-relation pairs
    for rel in no_relations:
        source_node = nodes[rel['source_idx']]
        target_node = nodes[rel['target_idx']]
        
        doc_pairs.append({
            'source_text': source_node['text'],
            'target_text': target_node['text'],
            'label': 2,  # 2=No-Relation
            'document': xml_file.name
        })
    
    # Only store if we have pairs for this document
    if doc_pairs:
        doc_to_pairs[xml_file.name] = doc_pairs

# Get documents that have pairs
valid_docs = list(doc_to_pairs.keys())
print(f"Documents with valid pairs: {len(valid_docs)}")

# Split documents into train and test
train_docs, test_docs = train_test_split(valid_docs, test_size=0.2, random_state=42)
print(f"Train documents: {len(train_docs)}")
print(f"Test documents: {len(test_docs)}")

# Flatten pairs by document
train_pairs = []
for doc in train_docs:
    train_pairs.extend(doc_to_pairs[doc])

test_pairs = []
for doc in test_docs:
    test_pairs.extend(doc_to_pairs[doc])

print(f"Train samples: {len(train_pairs)}")
print(f"Test samples: {len(test_pairs)}")

# Verify we have samples in both sets
if len(test_pairs) == 0:
    print("WARNING: No test samples! Using sample-level split instead of document-level split")
    # Fall back to sample-level split
    all_pairs = []
    for pairs in doc_to_pairs.values():
        all_pairs.extend(pairs)
    
    train_pairs, test_pairs = train_test_split(all_pairs, test_size=0.2, random_state=42)
    print(f"New split - Train: {len(train_pairs)}, Test: {len(test_pairs)}")


Processing XML files: 100%|██████████| 39/39 [00:00<00:00, 320.26it/s]

Sampled 77 No-Relation pairs from 9043 candidates
Sampled 99 No-Relation pairs from 9427 candidates
Sampled 56 No-Relation pairs from 2701 candidates
Sampled 98 No-Relation pairs from 5328 candidates
Sampled 64 No-Relation pairs from 2592 candidates
Sampled 114 No-Relation pairs from 9030 candidates
Sampled 121 No-Relation pairs from 12323 candidates
Sampled 74 No-Relation pairs from 2928 candidates
Sampled 65 No-Relation pairs from 5635 candidates
Sampled 43 No-Relation pairs from 1293 candidates
Sampled 35 No-Relation pairs from 839 candidates
Sampled 108 No-Relation pairs from 5786 candidates
Sampled 114 No-Relation pairs from 4083 candidates
Sampled 75 No-Relation pairs from 3961 candidates
Sampled 226 No-Relation pairs from 21324 candidates
Sampled 29 No-Relation pairs from 1031 candidates
Sampled 77 No-Relation pairs from 2494 candidates
Sampled 65 No-Relation pairs from 4627 candidates
Sampled 48 No-Relation pairs from 1592 candidates
Sampled 38 No-Relation pairs from 1298 candi

In [52]:
# Process in batches to avoid memory issues
def generate_pair_embeddings(pairs, batch_size=32):
    if not pairs:
        return None  # Return None for empty sets
    
    all_embeddings = []
    
    for i in tqdm(range(0, len(pairs), batch_size)):
        batch = pairs[i:i+batch_size]
        source_texts = [pair['source_text'] for pair in batch]
        target_texts = [pair['target_text'] for pair in batch]
        
        source_embs = generate_embeddings(source_texts)
        target_embs = generate_embeddings(target_texts)
        
        # Concatenate source and target embeddings
        for j in range(len(batch)):
            combined = torch.cat([source_embs[j], target_embs[j]], dim=0)
            all_embeddings.append(combined)
    
    return torch.stack(all_embeddings)

train_embeddings = generate_pair_embeddings(train_pairs)
test_embeddings = generate_pair_embeddings(test_pairs)

# Make sure we have embeddings before continuing
if train_embeddings is None or test_embeddings is None:
    raise ValueError("Failed to generate embeddings for both train and test sets")

print(f"Generated embeddings: Train {train_embeddings.shape}, Test {test_embeddings.shape}")


100%|██████████| 37/37 [00:16<00:00,  2.24it/s]

Generated embeddings: Train torch.Size([4162, 1536]), Test torch.Size([1166, 1536])


In [53]:
# Create a linear layer to increase dimensions to 1024
# expand_layer = nn.Linear(768*2, 2048)

# Apply linear transformation
# with torch.no_grad():
    # train_expanded = expand_layer(train_embeddings)
    # test_expanded = expand_layer(test_embeddings)


print("Applying UMAP dimensionality reduction...")
# Use UMAP to reduce to 512 dimensions
reducer = umap.UMAP(n_components=512, random_state=42)
train_reduced = reducer.fit_transform(train_embeddings.detach().numpy())
test_reduced = reducer.transform(test_embeddings.detach().numpy())

# Convert back to PyTorch tensors
train_reduced = torch.tensor(train_reduced, dtype=torch.float32)
test_reduced = torch.tensor(test_reduced, dtype=torch.float32)

print(f"Reduced dimensions: {train_reduced.shape}")

def plot_umap_embeddings(embeddings, labels, title):
    """Visualize embeddings with class-colored UMAP projection"""
    # reducer = umap.UMAP(n_components=2, random_state=42)
    reduced_2d = reducer.fit_transform(embeddings)
    
    # Convert numeric labels to meaningful names
    label_names = ['Support', 'Attack', 'No-Relation']
    colors = [label_names[label] for label in labels]
    
    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        x=reduced_2d[:,0], 
        y=reduced_2d[:,1], 
        hue=colors,
        palette='viridis',
        s=50,
        alpha=0.7,
        edgecolor='none'
    )
    plt.title(title)
    plt.xlabel('UMAP Dimension 1')
    plt.ylabel('UMAP Dimension 2')
    plt.legend(title='Relationship Type')
    plt.savefig(f"{title.replace(' ', '_').lower()}.png")
    plt.close()

# Get labels from training pairs
train_labels = [pair['label'] for pair in train_pairs]

try:
    # Plot original RoBERTa embeddings
    plot_umap_embeddings(
        train_embeddings.numpy(),
        train_labels,
        "768d Finetuned RoBERTa Embeddings (embedding layer)"
    )
    
    
    # Plot final UMAP-reduced embeddings
    plot_umap_embeddings(
        train_reduced.numpy(),
        train_labels,
        "512d UMAP-Reduced Embeddings Finetuned Roberta (embedding layer_)"
    )

except NameError as e:
    print(f"Missing variable: {e}. Run previous processing steps first")
except Exception as e:
    print(f"Visualization error: {str(e)}")


Applying UMAP dimensionality reduction...


/home/suyamoon/miniconda3/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Reduced dimensions: torch.Size([4162, 512])


In [8]:


# Define the MLP classifier
class RelationClassifier(nn.Module):
    def __init__(self, input_dim=512, hidden_dims=[256, 128]):
        super(RelationClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, 3))  # 3 classes: Support, Attack, No-Relation
        
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)

def calculate_balanced_class_weights(train_pairs):
    # Count instances of each class
    support_count = sum(1 for pair in train_pairs if pair['label'] == 0)
    attack_count = sum(1 for pair in train_pairs if pair['label'] == 1)
    no_relation_count = sum(1 for pair in train_pairs if pair['label'] == 2)
    
    total_samples = support_count + attack_count + no_relation_count
    
    # Calculate balanced weights
    support_w = total_samples / (3 * support_count)
    attack_w = total_samples / (3 * attack_count)
    no_rel_w = total_samples / (3 * no_relation_count)
    
    class_weights = torch.tensor([support_w, attack_w, no_rel_w], dtype=torch.float)
    
    print(f"Class weights - Support: {support_w:.2f}, Attack: {attack_w:.2f}, No Relation: {no_rel_w:.2f}")
    return class_weights

import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, reduction='none', weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss



In [9]:
# Custom dataset
class ArgumentPairDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings.detach().clone()  # Ensure detachment
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'embedding': self.embeddings[idx],
            'label': self.labels[idx]
        }

# Create datasets
train_labels = torch.tensor([pair['label'] for pair in train_pairs])
test_labels = torch.tensor([pair['label'] for pair in test_pairs])

train_dataset = ArgumentPairDataset(train_embeddings, train_labels)
test_dataset = ArgumentPairDataset(test_embeddings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    model = model.to(device)
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_loss = 0
        correct = 0
        total = 0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        progress_bar = tqdm(train_loader, desc="Training")
        
        for batch in progress_bar:
            embeddings = batch['embedding'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Statistics
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{100 * correct / total:.2f}%"
            })
        
        avg_train_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Train Loss: {avg_train_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")
        
        # Validation
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                embeddings = batch['embedding'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(embeddings)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print(f"Val Loss: {avg_val_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), OUTPUT_DIR / "best_model.pt")
            print("Saved new best model")
        
        # Generate confusion matrix every 5 epochs
        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
            cm = confusion_matrix(all_labels, all_preds)
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                       xticklabels=['Support', 'Attack', 'No-Relation'],
                       yticklabels=['Support', 'Attack', 'No-Relation'])
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.title(f'Confusion Matrix (Epoch {epoch+1})')
            plt.savefig(OUTPUT_DIR / f"confusion_matrix_epoch_{epoch+1}.png")
            plt.close()
    
    # Plot loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig(OUTPUT_DIR / "loss_curves.png")
    plt.close()
    
    return train_losses, val_losses


In [10]:
label_counts = Counter([pair['label'] for pair in train_pairs])
print(f"Class distribution - Support: {label_counts[0]}, Attack: {label_counts[1]}, No-Relation: {label_counts[2]}")


Class distribution - Support: 1782, Attack: 135, No-Relation: 2457


In [11]:
# Calculate class weights based on training data
class_weights = calculate_balanced_class_weights(train_pairs)
class_weights = class_weights.to(device)

# Initialize model, criterion, and optimizer
model = RelationClassifier(input_dim=768*2, hidden_dims=[1024, 512])
criterion = FocalLoss(weight=class_weights, gamma=2.0)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)


Class weights - Support: 0.82, Attack: 10.80, No Relation: 0.59


In [12]:

# Train model
print("Training model...")
train_losses, val_losses = train_model(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=20
)

# Load best model for final evaluation
model.load_state_dict(torch.load(OUTPUT_DIR / "best_model.pt"))
model.eval()

# Final evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Final evaluation"):
        embeddings = batch['embedding'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(embeddings)
        _, predicted = torch.max(outputs.data, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate final confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Support', 'Attack', 'No-Relation'],
           yticklabels=['Support', 'Attack', 'No-Relation'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Final Confusion Matrix')
plt.savefig(OUTPUT_DIR / "final_confusion_matrix.png")
plt.close()

# Print final classification report
report = classification_report(
    all_labels, all_preds,
    target_names=['Support', 'Attack', 'No-Relation'],
    digits=4
)
print("Final Classification Report:")
print(report)

# Save the report
with open(OUTPUT_DIR / "classification_report.txt", "w") as f:
    f.write(report)

print(f"All results saved to {OUTPUT_DIR}")


Training model...
Epoch 1/20


Training: 100%|██████████| 137/137 [00:02<00:00, 58.55it/s, loss=0.2002, acc=52.35%]


Train Loss: 0.4994, Accuracy: 52.35%


Validation: 100%|██████████| 34/34 [00:00<00:00, 539.10it/s]


Val Loss: 0.1524, Accuracy: 64.66%
Saved new best model
Epoch 2/20


...

Train Loss: 0.3227, Accuracy: 55.14%


Validation: 100%|██████████| 34/34 [00:00<00:00, 480.00it/s]


Val Loss: 0.1388, Accuracy: 56.03%


Final evaluation: 100%|██████████| 34/34 [00:00<00:00, 651.56it/s]


Final Classification Report:
              precision    recall  f1-score   support

     Support     0.9885    0.8448    0.9110       509
      Attack     0.0476    0.4000    0.0851        10
 No-Relation     0.9964    0.9964    0.9964       559

    accuracy                         0.9193      1078
   macro avg     0.6775    0.7471    0.6642      1078
weighted avg     0.9839    0.9193    0.9476      1078

All results saved to classifier_results
